<a href="https://colab.research.google.com/github/pvpogorelova/metrics_24_25/blob/main/Sem_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd # работа с таблицами
import numpy as np # математические функции и матрицы
import matplotlib.pyplot as plt # графики
import seaborn as sns # еще более классные графики
import statsmodels.api as sm # стандартные регрессионные модели
import statsmodels.formula.api as smf # аналогичные модели с синтаксисом в стиле R
import statsmodels.graphics.gofplots as gf # визуализация моделей
import statsmodels.discrete.discrete_model # дискретные модели
from statsmodels.stats.outliers_influence import summary_table # работа с выбросами
from scipy.stats import shapiro # тест Шапиро – Уилка

In [ ]:
# При желании можем настроить графики по своему вкусу, изменив стиль и другие параметры шрифтов и графиков
# (https://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html)
plt.style.use('seaborn')
plt.rc('font', size=14)
plt.rc('figure', titlesize=15)
plt.rc('axes', labelsize=15)
plt.rc('axes', titlesize=15)

<ipython-input-2-9bd5fa555a4a>:3: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [7]:
# Импортируем данные о квартирах в Москве
data = pd.read_excel('dataflats.xlsx')

In [8]:
data

,n,price,totsp,livesp,kitsp,dist,metrdist,walk,brick,floor
0,1,81.0,58,40,6.0,12.5,7,1,1,1
1,2,75.0,44,28,6.0,13.5,7,1,0,1
2,3,128.0,70,42,6.0,14.5,3,1,1,1
3,4,95.0,61,37,6.0,13.5,7,1,0,1
4,5,330.0,104,60,11.0,10.5,7,0,1,1
...,...,...,...,...,...,...,...,...,...,...
2035,2036,110.0,77,45,10.0,12.0,5,0,0,1
2036,2037,95.0,60,43,6.0,9.0,5,0,0,1
2037,2038,95.0,60,46,5.0,10.5,5,1,0,1
2038,2039,129.0,76,48,10.0,12.5,5,0,0,1


In [9]:
# Удалим строки, в которых есть пропуск хотя бы для одного столбца
data.dropna(inplace=True)

In [10]:
# Добавим в набор данных новую переменную price_sq (стоимость 1 кв м квартиры в Москве)
data['price_sq'] = data['price']/data['totsp']

In [11]:
data

,n,price,totsp,livesp,kitsp,dist,metrdist,walk,brick,floor,price_sq
0,1,81.0,58,40,6.0,12.5,7,1,1,1,1.396552
1,2,75.0,44,28,6.0,13.5,7,1,0,1,1.704545
2,3,128.0,70,42,6.0,14.5,3,1,1,1,1.828571
3,4,95.0,61,37,6.0,13.5,7,1,0,1,1.557377
4,5,330.0,104,60,11.0,10.5,7,0,1,1,3.173077
...,...,...,...,...,...,...,...,...,...,...,...
2035,2036,110.0,77,45,10.0,12.0,5,0,0,1,1.428571
2036,2037,95.0,60,43,6.0,9.0,5,0,0,1,1.583333
2037,2038,95.0,60,46,5.0,10.5,5,1,0,1,1.583333
2038,2039,129.0,76,48,10.0,12.5,5,0,0,1,1.697368


In [12]:
# a) Оценим регрессионные модели

# С помощью МНК оценим модель регрессии для стоимости 1 кв м квартиры, используя в качестве объясняющих переменных
# livesp (жилая площадь) и dist (расстояние до центра в км)
ols_1 = smf.ols('price_sq ~ livesp + dist', data=data).fit()
print(ols_1.summary())

                            OLS Regression Results                            
Dep. Variable:               price_sq   R-squared:                       0.279
Model:                            OLS   Adj. R-squared:                  0.278
Method:                 Least Squares   F-statistic:                     393.7
Date:                Fri, 04 Oct 2024   Prob (F-statistic):          2.94e-145
Time:                        09:44:09   Log-Likelihood:                -717.45
No. Observations:                2038   AIC:                             1441.
Df Residuals:                    2035   BIC:                             1458.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.5395      0.057     26.994      0.0

In [ ]:
# Спрогнозируем модельные значения для имеющихся наблюдений
data['yhat'] = ols_1.fittedvalues

In [ ]:
data

,price,totsp,livesp,kitsp,dist,metrdist,walk,brick,floor,price_sq,yhat
0,81,58,40,6.0,12.5,7,1,1,1,1.396552,1.554974
1,75,44,28,6.0,13.5,7,1,0,1,1.704545,1.331456
2,128,70,42,6.0,14.5,3,1,1,1,1.828571,1.490986
3,95,61,37,6.0,13.5,7,1,0,1,1.557377,1.464049
4,330,104,60,11.0,10.5,7,0,1,1,3.173077,1.943079
...,...,...,...,...,...,...,...,...,...,...,...
2035,110,77,45,10.0,12.0,5,0,0,1,1.428571,1.652000
2036,95,60,43,6.0,9.0,5,0,0,1,1.583333,1.762715
2037,95,60,46,5.0,10.5,5,1,0,1,1.583333,1.736823
2038,129,76,48,10.0,12.5,5,0,0,1,1.697368,1.672834


In [ ]:
# С помощью МНК оценим альтернативную модель регрессии, включив дополнительно переменную metrdist
# (расстояние до ближайшей станции метро (в минутах))
ols_2 = smf.ols('price_sq ~ livesp + dist + metrdist', data=data).fit()
print(ols_2.summary())

                            OLS Regression Results                            
Dep. Variable:               price_sq   R-squared:                       0.300
Model:                            OLS   Adj. R-squared:                  0.299
Method:                 Least Squares   F-statistic:                     291.3
Date:                Thu, 03 Oct 2024   Prob (F-statistic):          2.53e-157
Time:                        15:10:25   Log-Likelihood:                -690.65
No. Observations:                2040   AIC:                             1389.
Df Residuals:                    2036   BIC:                             1412.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.6760      0.058     28.769      0.0

In [ ]:
# b) Проверим гипотезу о незначимости моделей в целом с помощью F-теста
# Для первой модели:
hypotheses1 = '(livesp = dist = 0)' # формулируем гипотезу H_0
print(ols_1.f_test(r_matrix = hypotheses1)) # тестируем гипотезу с помощью F-теста

# Для второй модели
hypotheses2 = '(livesp = dist = metrdist = 0)' # формулируем гипотезу H_0
print(ols_2.f_test(r_matrix = hypotheses2)) # тестируем гипотезу с помощью F-теста

<F test: F=390.1735423108271, p=3.5072204862173017e-144, df_denom=2.04e+03, df_num=2>
<F test: F=291.30023962278113, p=2.532626535653841e-157, df_denom=2.04e+03, df_num=3>


In [ ]:
# e) Построим 90%-й доверительные интервалы для оценок коэффициентов моделей ols_1 и ols_2
print(ols_1.conf_int(alpha = 0.1))
print(ols_2.conf_int(alpha = 0.1))

                  0         1
Intercept  1.455753  1.643755
livesp     0.013106  0.016359
dist      -0.050530 -0.042924
                  0         1
Intercept  1.580118  1.771852
livesp     0.012864  0.016066
dist      -0.048775 -0.041260
metrdist  -0.019606 -0.013080


In [ ]:
# f) Проверим гипотезу о равенстве коэффициента при регрессоре dist -0.1 в моделях ols_1 и ols_2
hypotheses = '(dist = -0.1)' # формулируем гипотезу H_0
# Первая модель
print(ols_1.t_test(r_matrix = hypotheses)) # тестируем гипотезу с помощью t-теста
print(ols_1.f_test(r_matrix = hypotheses)) # тестируем гипотезу с помощью F-теста
# Вторая модель
print(ols_2.t_test(r_matrix = hypotheses)) # тестируем гипотезу с помощью t-теста
print(ols_2.f_test(r_matrix = hypotheses)) # тестируем гипотезу с помощью F-теста

                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -0.0467      0.002     23.052      0.000      -0.051      -0.042
<F test: F=531.387876214532, p=1.139550472153157e-104, df_denom=2.04e+03, df_num=1>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -0.0450      0.002     24.079      0.000      -0.049      -0.041
<F test: F=579.7999923409784, p=6.14148936798743e-113, df_denom=2.04e+03, df_num=1>


In [ ]:
# g) Проверим гипотезу
hypotheses = '(livesp+dist=0)' # формулируем гипотезу H_0
print(ols_1.t_test(r_matrix = hypotheses)) # тестируем гипотезу с помощью t-теста
print(ols_2.t_test(r_matrix = hypotheses)) # тестируем гипотезу с помощью t-теста

                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -0.0320      0.003    -11.909      0.000      -0.037      -0.027
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0            -0.0306      0.003    -11.532      0.000      -0.036      -0.025


In [ ]:
# h) Переоценим регрессию для шкалированных данных
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

data_scale=data.copy()
data_scale=pd.DataFrame(scaler.fit_transform(data_scale), columns=data_scale.columns)
data_scale

,price,totsp,livesp,kitsp,dist,metrdist,walk,brick,floor,price_sq,yhat
0,-0.895412,-0.997657,-0.802955,-1.040579,0.439834,-0.292860,0.676893,1.447619,0.514513,-0.791322,-0.761859
1,-1.011115,-1.923598,-2.323403,-1.040579,0.736156,-0.292860,0.676893,-0.690790,0.514513,-0.032401,-1.808366
2,0.010928,-0.203992,-0.549547,-1.040579,1.032478,-1.341452,0.676893,1.447619,0.514513,0.273208,-1.061450
3,-0.625439,-0.799241,-1.183067,-1.040579,0.736156,-0.292860,0.676893,-0.690790,0.514513,-0.395036,-1.187566
4,3.906261,2.044723,1.731124,0.754129,-0.152809,-0.292860,-1.477338,1.447619,0.514513,3.586173,1.055245
...,...,...,...,...,...,...,...,...,...,...,...
2035,-0.336181,0.258978,-0.169435,0.395188,0.291673,-0.817156,-1.477338,-0.690790,0.514513,-0.712423,-0.307583
2036,-0.625439,-0.865379,-0.422843,-1.040579,-0.597291,-0.817156,-1.477338,-0.690790,0.514513,-0.331077,0.210783
2037,-0.625439,-0.865379,-0.042732,-1.399521,-0.152809,-0.817156,0.676893,-0.690790,0.514513,-0.331077,0.089555
2038,0.030211,0.192840,0.210676,0.395188,0.439834,-0.817156,-1.477338,-0.690790,0.514513,-0.050086,-0.210036


In [ ]:
# Оценим модель регрессии ols_2 для шкалированных показателей
ols_2_scale = smf.ols('price_sq ~ livesp + dist + metrdist', data=data_scale).fit()
print(ols_2_scale.summary())

                            OLS Regression Results                            
Dep. Variable:               price_sq   R-squared:                       0.300
Model:                            OLS   Adj. R-squared:                  0.299
Method:                 Least Squares   F-statistic:                     291.3
Date:                Thu, 03 Oct 2024   Prob (F-statistic):          2.53e-157
Time:                        15:14:34   Log-Likelihood:                -2530.4
No. Observations:                2040   AIC:                             5069.
Df Residuals:                    2036   BIC:                             5091.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -4.163e-17      0.019  -2.25e-15      1.0